# Connecting to the board

`%serialconnect` should automatically detect the port, but it doesn't work you can provide the port and baudrate as parameters to this magic function

In [2]:
# %serialconnect
%serialconnect --port=/dev/tty.usbmodem3660384B30362 --baud=115200

Connecting to --port=/dev/tty.usbmodem3660384B30362 --baud=115200 
Ready.


`help('modules')` is a handy function that returns a list of available modules
Our build has a few extra (or extended) modules: 
- `bitcoin` - written in python and contains all necessary functions to build a hardware wallet
- `hashlib` - adds support of sha512, ripemd160 and a few extra one-liners like `pbkdf2_hmac_sha512` and `hmac_sha512`
- `display` - allows to init and update the display, all the gui stuff should be done with `lvgl`
- `lvgl` - micropython bindings to [littlevgl](https://littlevgl.com/) library. It is a very powerful and optimized GUI library with plenty of widgets and advanced features like anti-aliasing, custom fonts etc.
- `qrcode` - binding to C library that generates QR codes from a string

In [3]:
help('modules')

__main__          bitcoin/transaction                 qrcode            ujson
_onewire          builtins          secp256k1         umachine
bitcoin/__init__  cmath             stm               uos
bitcoin/base58    display           sys               urandom
bitcoin/bech32    framebuf          uarray            ure
bitcoin/bip32     gc                ubinascii         uselect
bitcoin/bip39     hashlib           ucollections      usocket
bitcoin/compact   lvgl              ucryptolib        ussl
bitcoin/ec        main              uctypes           ustruct
bitcoin/hashes    math              uerrno            utime
bitcoin/networks  micropython       uhashlib          utimeq
bitcoin/psbt      network           uheapq            uzlib
bitcoin/script    pyb               uio
Plus any modules on the filesystem


In this part of the tutorial we are interested in `pyb` module. It gives you an interface to communicate with hardware peripherals, in particular with LEDs and a switch (the blue button on the back of the board)

# Blinking with LEDs

`pyb.LED` is a class that allows you to turn LEDs on and off. There are 4 LEDs on the board (right above the screen)

Let's turn them on!

In [4]:
import pyb

# list of LEDs:
leds = [pyb.LED(i) for i in range(1,5)]

# turn on every LED
for led in leds:
    led.on()

Now let's make them roll. We will shine one LED at a time and move to the next one after 100 ms.

As it we have an infinite loop here we will need to interrupt the process when we are bored to look at rolling LEDs. In Jupyter you can do it from the top menu: `Kernel->Interrupt`

In [5]:
import time
cur = 0 # index of the LED we will turn on
while True:
    for i,led in enumerate(leds):
        # turn on current led
        if i==cur:
            led.on()
        else: # turn off every other
            led.off()
    cur = (cur+1) % len(leds)
    time.sleep_ms(100)



*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
KeyboardInterrupt: 


# Schedule and Timer

Now let's make it non-blocking. This will be important later when we will start wringing our GUI.

The board also has a `pyb.Timer` class that can call a function with a certain period. This is exactly what we need for our LEDs. 

The only thing you need to remember is that this function works in the interrupt mode that blocks all other processes. This function should be as small as possible, otherwise it is way better not to call this function right away but to add it to the queue. Functions in the queue will be processes "as soon as possible" during normal operation. 

Micropython has a special method to add a function to this queue: `micropython.schedule`. If the queue is full it will raise an error, but we don't care if one of the function calls will be skipped, so we will `try` it.

In [6]:
import micropython
# we will change `step` variable 
# to change the direction of the roll
step = 1
# this is our counter
cur = 0
def roll(t):
    """Roll to the next LED"""
    global cur
    for i,led in enumerate(leds):
        # turn on current led
        if i==cur:
            led.on()
        else: # turn off every other
            led.off()
    cur = (cur+step) % len(leds)

def schedule(t):
    """Try to schedule an LED update"""
    try:
        micropython.schedule(roll, None)
    except:
        pass

timer = pyb.Timer(4) # timer 4
timer.init(freq=10)# 10Hz - 100 ms per tick
timer.callback(schedule)

In [7]:
# we can interactively change the `step` now and reverse the direction
step = len(leds)-1

Or we can use a button to control the direction:

In [11]:
sw = pyb.Switch()

def change_direction():
    global step
    step = 1 if step > 1 else len(leds)-1
sw.callback(change_direction)

In [12]:
# In order to stop this LED dance we need to
# unregister callback and deinit the timer
timer.callback(None)
timer.deinit()
for led in leds:
    led.off()

Now when we spent some time with LEDs let's move on and write a small GUI that controls the LEDs.

Also check out the `main.py` file in this folder. You can copy it to the board and it will run this script after reset (black button)